In [2]:
import numpy as np
import math

---

**Functions**

In [ ]:
def sphere(x):
    """
        Sphere function - a simple test function for optimization
        f(x) = sum(x_i^2)
        Global minimum: f(0,...,0) = 0
    """
    return np.sum(x**2)

def schwefel(x):
    """
        Schwefel 2.21 function
        f(x) = max(|xi|) for i=1,...,n
        Global minimum: f(0,...,0) = 0
        Search space: [-100, 100]
    """
    return np.max(np.abs(x))

def ackley(x, a=20, b=0.2, c=2*np.pi):
    """
        Ackley function
        f(x) = -a * exp(-b * sqrt(1/d * sum(x_i^2))) - exp(1/d * sum(cos(c*x_i))) + a + exp(1)
        where:
        a = 20, b = 0.2, c = 2π are standard values
        d is the dimension of x
        Global minimum: f(0,...,0) = 0
        Search space: [-32, 32]
    """
    d = len(x)
    sum_sq = np.sum(x**2)
    sum_cos = np.sum(np.cos(c * x))
    
    term1 = -a * np.exp(-b * np.sqrt(sum_sq/d))
    term2 = -np.exp(sum_cos/d)
    
    return term1 + term2 + a + np.exp(1)

def griewank(x):
    """
        Griewank function
        f(x) = 1 + (1/4000)*sum(x_i^2) - prod(cos(x_i/sqrt(i)))
        Global minimum: f(0,...,0) = 0
        Search space: [-600, 600]
        
        The function has many widespread local minima, which are regularly distributed.
        The product term makes the function non-separable.
    """
    sum_term = np.sum(x**2) / 4000
    
    indices = np.arange(1, len(x) + 1)
    prod_term = np.prod(np.cos(x / np.sqrt(indices)))
    
    return 1 + sum_term - prod_term

---
**Helpers**

In [4]:
def fitness(x, func):
    """
        Calculates the fitness value for a solution using the given function
    """
    return func(x)

def init_pop(pop_size, nr_dim, bounds):
    """
        Initialize population with random solutions within the given bounds
    """
    return np.random.uniform(bounds[0], bounds[1], size=(pop_size, nr_dim))

---
**Whale optimization algorithm**

In [5]:
def woa(pop_size: int, spiral_shape_const: float, nr_dim: int, func, bounds: tuple, nr_iter: int):
    """
        Performs whale optimization algorithm to minimize the value of a given function func
        in: 
        pop_size - int, population size
        spiral_shape_const - float, controls the tightness of the log spiral
        nr_dim - int, number of dimensions of the function input
        func - function to minimize
        bounds - tuple, bounds of the values of the function
        nr_iter - int, maximum number of iterations
    """
    curr_pop = init_pop(pop_size, nr_dim, bounds)
    a = 2

    # find the initial best solution
    global_best = min(curr_pop, key=lambda x : fitness(x, func))
    global_best_fitness = fitness(global_best, func)

    for i in range(nr_iter):
        # update a linearly from 2 to 0
        a -= 2 * i / nr_iter

        for j in range(pop_size):
            whale = curr_pop[j]

            # generate random numbers for encircling mechanism
            r1 = np.random.uniform(0, 1)
            r2 = np.random.uniform(0, 1)

            A = 2 * a * r1 - a # determines search area size
            C = 2 * r2 # random weight for best solution

            if np.random.uniform(0, 1) < 0.5:
                if abs(A) < 1: # exploitation: shrinking circle
                    D = np.abs(C * global_best - whale)
                    whale = global_best - A * D
                else: # exploration: random search
                    random_whale = curr_pop[np.random.randint(0, pop_size)]
                    D = np.abs(C * random_whale - whale)
                    whale = random_whale - A * D
            else: # exploitation: spiral update
                D = np.abs(global_best - whale)
                spiral_param = np.random.uniform(-1, 1)
                whale = D * math.exp(spiral_shape_const * spiral_param) * math.cos(2 * math.pi * spiral_param) + global_best
            
            # bound constraints
            whale = np.clip(whale, bounds[0], bounds[1])
            whale_fitness = fitness(whale, func)

            if whale_fitness < global_best_fitness:
                global_best = whale
                global_best_fitness = whale_fitness

            curr_pop[j] = whale

    return global_best, global_best_fitness

---
**Test**

In [7]:
pop_size = 30
spiral_const = 1.0
dimensions = 2
bounds = (-5, 5)
iterations = 100

best_solution, best_fitness = woa(
    pop_size=pop_size,
    spiral_shape_const=spiral_const,
    nr_dim=dimensions,
    func=sphere,
    bounds=bounds,
    nr_iter=iterations
)
print(f"Best solution found for sphere: {best_solution}")
print(f"Best fitness value for sphere: {best_fitness}\n")

best_solution, best_fitness = woa(
    pop_size=pop_size,
    spiral_shape_const=spiral_const,
    nr_dim=dimensions,
    func=schwefel,
    bounds=bounds,
    nr_iter=iterations
)
print(f"Best solution found for schwefel: {best_solution}")
print(f"Best fitness value for schwefel: {best_fitness}\n")

best_solution, best_fitness = woa(
    pop_size=pop_size,
    spiral_shape_const=spiral_const,
    nr_dim=dimensions,
    func=ackley,
    bounds=bounds,
    nr_iter=iterations
)
print(f"Best solution found for ackley: {best_solution}")
print(f"Best fitness value for ackley: {best_fitness}\n")

best_solution, best_fitness = woa(
    pop_size=pop_size,
    spiral_shape_const=spiral_const,
    nr_dim=dimensions,
    func=griewank,
    bounds=bounds,
    nr_iter=iterations
)
print(f"Best solution found for griewank: {best_solution}")
print(f"Best fitness value for griewank: {best_fitness}\n")

Best solution found for sphere: [-9.03252067e-09  7.70905507e-08]
Best fitness value for sphere: 6.024539443505664e-15

Best solution found for schwefel: [-2.08562169e-05 -5.46292415e-05]
Best fitness value for schwefel: 5.462924145521297e-05

Best solution found for ackley: [ 1.40556449e-07 -3.54705973e-08]
Best fitness value for ackley: 4.1001790807015936e-07

Best solution found for griewank: [3.13989364 4.44150067]
Best fitness value for griewank: 0.0073983860649956545



---
**Improved whale optimization algorithm**

In [8]:
def iwoa(pop_size: int, nr_dim: int, func, bounds: tuple, nr_iter: int, F: float = 0.5, CR: float = 0.7):
    """
        Improved Whale Optimization Algorithm (IWOA)
        
        pop_size : int, population size (number of whales)
        nr_dim : int, number of dimensions
        func : callable, objective function to minimize
        bounds : tuple, (lower_bound, upper_bound) for solution space
        nr_iter : int, maximum number of iterations
        F : float, differential weight (mutation factor) for DE operator
        CR : float, crossover probability for DE operator
    """
    # initialize population
    population = init_pop(pop_size, nr_dim, bounds)
    fitness_values = np.array([fitness(x, func) for x in population])
    
    # find the initial best solution
    best_idx = np.argmin(fitness_values)
    X_star = population[best_idx].copy()
    best_fitness = fitness_values[best_idx]
    
    for t in range(nr_iter):
        # calculate adaptive parameter k
        k = 1 - (t / nr_iter)
        a = 2 - t * (2/nr_iter)
        
        for i in range(pop_size):
            # select three random indices different from i
            r1, r2, r3 = np.random.choice([j for j in range(pop_size) if j != i], 3, replace=False)
            
            p = np.random.uniform(0, 1) # random number for probability check
            
            # initialize offspring
            U = np.zeros(nr_dim)
            
            # random index for DE
            j_rand = np.random.randint(0, nr_dim)
            
            for j in range(nr_dim):
                if p <= k:  # exploration phase
                    if np.random.uniform(0, 1) <= CR or j == j_rand:
                        # DE mutation
                        U[j] = X_star[j] + F * (population[r2][j] - population[r3][j])
                    else:
                        # select random whale
                        x_rand = population[np.random.randint(0, pop_size)]
                        D = abs(x_rand[j] - population[i][j])
                        U[j] = x_rand[j] - a * D
                else:  # exploitation phase
                    if np.random.uniform(0, 1) <= 0.5:
                        # shrinking encircling
                        D = abs(X_star[j] - population[i][j])
                        U[j] = X_star[j] - a * D
                    else:
                        # spiral update
                        D = abs(X_star[j] - population[i][j])
                        l = np.random.uniform(-1, 1)
                        U[j] = D * math.exp(1 * l) * math.cos(2 * math.pi * l) + X_star[j]
            
            U = np.clip(U, bounds[0], bounds[1])
            U_fitness = fitness(U, func)
            
            # selection: elitism
            if U_fitness < fitness_values[i]:
                population[i] = U
                fitness_values[i] = U_fitness
                
                if U_fitness < best_fitness:
                    X_star = U.copy()
                    best_fitness = U_fitness
        
    
    return X_star, best_fitness

---
**Test**

In [11]:
pop_size = 30
spiral_const = 1.0
dimensions = 2
bounds = (-5, 5)
iterations = 100
F = 0.5
CR = 0.7 

best_solution, best_fitness = iwoa(
    pop_size=pop_size,
    nr_dim=dimensions,
    func=sphere,
    bounds=bounds,
    nr_iter=iterations,
    F=F,
    CR=CR
)
print(f"Best solution found for sphere: {best_solution}")
print(f"Best fitness value for sphere: {best_fitness}\n")

best_solution, best_fitness = iwoa(
    pop_size=pop_size,
    nr_dim=dimensions,
    func=schwefel,
    bounds=bounds,
    nr_iter=iterations,
    F=F,
    CR=CR
)
print(f"Best solution found for schwefel: {best_solution}")
print(f"Best fitness value for schwefel: {best_fitness}\n")

best_solution, best_fitness = iwoa(
    pop_size=pop_size,
    nr_dim=dimensions,
    func=ackley,
    bounds=bounds,
    nr_iter=iterations,
    F=F,
    CR=CR
)
print(f"Best solution found for ackley: {best_solution}")
print(f"Best fitness value for ackley: {best_fitness}\n")

best_solution, best_fitness = iwoa(
    pop_size=pop_size,
    nr_dim=dimensions,
    func=griewank,
    bounds=bounds,
    nr_iter=iterations,
    F=F,
    CR=CR
)
print(f"Best solution found for griewank: {best_solution}")
print(f"Best fitness value for griewank: {best_fitness}\n")

Best solution found for sphere: [-3.57045151e-26 -4.13669285e-26]
Best fitness value for sphere: 2.9860351687324077e-51

Best solution found for schwefel: [-1.79455081e-26 -2.80482876e-26]
Best fitness value for schwefel: 2.8048287563050374e-26

Best solution found for ackley: [-1.90716276e-16  7.13576006e-17]
Best fitness value for ackley: 4.440892098500626e-16

Best solution found for griewank: [-3.14002263 -4.43844448]
Best fitness value for griewank: 0.007396040334114895

